In [1]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [2]:
import pandas as pd

origin_multihop_rag=pd.read_csv("data/benchmark_datasets/multihop_rag/corpus/corpus_chunks.csv")
final_entities=pd.read_csv("export/multihop_rag/multihop_rag_entities_by_chatgpt4o.csv")
final_relationships=pd.read_csv("export/multihop_rag/multihop_rag_relationships_by_chatgpt4o.csv")

# Multihop-RAG 原始数据

In [3]:
origin_multihop_rag.head(10)

,id,category,author,published_at,title,source,chunk,token_count
0,0,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"table of contents table of contents echo , fir...",300
1,1,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,amazon ' s invite - only deals for prime membe...,304
2,2,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"99 ) , our new favorite ipad deal at amazon is...",317
3,3,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"on the outside , the go 3 is definitely giving...",386
4,4,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,"etf provider betashares , which manages $ 30 b...",314
5,5,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,4 per cent . the lowdown novus capital senior ...,309
6,6,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,55 per cent from 4 . 54 per cent late on monda...,304
7,7,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,"on wall street , the vast majority of stocks f...",307
8,8,business,NaN,2023-10-06 21:31:00+00:00,Amazon sellers sound off on the FTC's 'long-ov...,Cnbc | World Business News Leader,a worker sorts out parcels in the outbound doc...,323
9,9,business,NaN,2023-10-06 21:31:00+00:00,Amazon sellers sound off on the FTC's 'long-ov...,Cnbc | World Business News Leader,he opted for the latter . fred ruckel ' s comp...,308


# Multihop-RAG Extracted 数据展示

In [4]:
# 设置pandas显示选项，增加最大列宽度
pd.set_option('display.max_colwidth', 200)  # 将200替换为你想要的长度
# 筛选 entity_name 长度小于30且 description 长度大于150 的行
filtered_df = final_entities[
    (final_entities['entity_name'].str.len() < 30) & 
    (final_entities['entity_name'].str.len() > 10) & 
    (final_entities['description'].str.len() > 50)
]

# 查看筛选后的数据框
filtered_df.head(10)

,entity_name,entity_type,description,entity_id,corpus_id
1,CYBER MONDAY,EVENT,"Cyber Monday is a significant shopping event that occurs on the Monday after Thanksgiving, featuring various online sales and promotions.",552a9c2e-cfcf-45a5-8443-49924faba368,0
2,BLACK FRIDAY,EVENT,"Black Friday is a shopping holiday that takes place the day after Thanksgiving, marked by major discounts and sales.",a35c613e-36cb-434f-bf78-5b0c60d97a99,0
3,"NOVEMBER 27, 2023",TIME,"November 27, 2023, is the date when Cyber Monday is being observed this year.",5bd9ea91-f8f7-448b-87f9-c6c8e6cfc211,0
9,AUDIBLE PREMIUM PLUS,PRODUCT,Audible Premium Plus is a subscription service for audiobooks and spoken word content offered by Amazon.,8855fc52-1b06-44c6-b1fc-f906e265a420,1
11,BLACK FRIDAY,TIME,"Black Friday is the day following Thanksgiving in the United States, often associated with significant retail sales and discounts.",abf44de0-d3d1-4954-bbfe-d052192651d6,1
12,CYBER MONDAY,TIME,"Cyber Monday is the Monday following Thanksgiving, recognized for online shopping deals and promotions.",454fa3ac-c6e3-4020-bb20-07a195ebf288,1
15,A14 BIONIC CHIP,CONCEPT,"The A14 Bionic chip is a powerful processor used in the 10th generation iPad, designed to enhance performance for multitasking.",f15854fe-0f71-4402-adf3-ba828724fbfc,2
17,MICROSOFT SURFACE LAPTOP GO 3,PRODUCT,The Microsoft Surface Laptop Go 3 is a lightweight laptop that features an Intel Core i5 processor and is marketed as a budget-friendly option.,4eb53121-49bc-4052-b618-6de823afd5dd,2
18,CYBER MONDAY,EVENT,"Cyber Monday is an online shopping event that follows Black Friday, known for significant discounts and sales on electronics and other products.",4e48707d-9916-4d25-ad1d-71d32ef2e9ba,2
20,MACBOOK AIR,PRODUCT,The MacBook Air is a traditional laptop known for its lightweight and sleek design.,ac6f548e-b18f-4c75-9bff-c402068d5da3,3


In [5]:
# 设置pandas显示选项，增加最大列宽度
pd.set_option('display.max_colwidth', 200)  # 将200替换为你想要的长度
# 筛选 entity_name 长度小于30且 description 长度大于150 的行
filtered_df = final_relationships[
    (final_relationships['source_entity'].str.len() < 30) & 
    (final_relationships['source_entity'].str.len() > 5) & 
    (final_relationships['target_entity'].str.len() < 30) & 
    (final_relationships['target_entity'].str.len() > 5) & 
    (final_relationships['relationship_description'].str.len() > 50)
]

# 查看筛选后的数据框
filtered_df.head(10)

,source_entity,target_entity,relationship_description,relationship_strength,source_entity_id,target_entity_id,corpus_id
0,AMAZON,CYBER MONDAY,"Amazon is hosting numerous deals during Cyber Monday, extending its sales from Black Friday.",9,6a870a77-b48d-4cef-b9b9-59e9e262e8ea,552a9c2e-cfcf-45a5-8443-49924faba368,0
1,AMAZON,BLACK FRIDAY,"Amazon began its Black Friday sale a week ahead of schedule, marking it as a key event for the retailer.",8,6a870a77-b48d-4cef-b9b9-59e9e262e8ea,a35c613e-36cb-434f-bf78-5b0c60d97a99,0
2,CYBER MONDAY,BLACK FRIDAY,"Cyber Monday follows Black Friday, with many deals carried over from the previous shopping event.",7,552a9c2e-cfcf-45a5-8443-49924faba368,a35c613e-36cb-434f-bf78-5b0c60d97a99,0
3,"NOVEMBER 27, 2023",CYBER MONDAY,"November 27, 2023, is the date that marks the celebration of Cyber Monday.",10,5bd9ea91-f8f7-448b-87f9-c6c8e6cfc211,552a9c2e-cfcf-45a5-8443-49924faba368,0
4,AMAZON,ECHO SHOW,"Amazon produces the Echo Show, integrating it with its Alexa virtual assistant for enhanced functionality.",8,59438462-3844-4ccf-abc6-04d192abe29d,b863c742-eb42-43ff-809e-8f38fef43499,1
5,AMAZON,FIRE TV,"Amazon develops and sells the Fire TV, a product that allows streaming of various media.",8,59438462-3844-4ccf-abc6-04d192abe29d,b163a802-8fa6-401f-8223-a5a0559e2e3f,1
6,AMAZON,KINDLE,"Amazon designs and markets the Kindle, a popular e-reader in the digital book market.",8,59438462-3844-4ccf-abc6-04d192abe29d,e038d3a1-d123-41d8-b792-f3a6624c353d,1
7,AMAZON,AUDIBLE PREMIUM PLUS,"Audible Premium Plus is a subscription service offered by Amazon, providing access to audiobooks.",8,59438462-3844-4ccf-abc6-04d192abe29d,8855fc52-1b06-44c6-b1fc-f906e265a420,1
8,AMAZON,BLACK FRIDAY,Amazon participates in Black Friday sales with significant discounts on its products.,7,59438462-3844-4ccf-abc6-04d192abe29d,abf44de0-d3d1-4954-bbfe-d052192651d6,1
9,AMAZON,CYBER MONDAY,"Amazon offers promotions and deals on Cyber Monday, targeting online shoppers.",7,59438462-3844-4ccf-abc6-04d192abe29d,454fa3ac-c6e3-4020-bb20-07a195ebf288,1


# 加载Embedding数据

In [2]:
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

final_entities=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_entities_with_embeddings_by_chatgpt4o.csv",['entity_embedding'])
final_entities

,entity_name,entity_type,description,entity_id,corpus_id,entity_embedding
0,AMAZON,ORGANIZATION,Amazon is a retail giant known for its extensi...,0,0,"[0.025474460795521736, -0.019272109493613243, ..."
1,CYBER MONDAY,EVENT,Cyber Monday is a significant shopping event t...,1,0,"[0.008677905425429344, -0.012837450951337814, ..."
2,BLACK FRIDAY,EVENT,Black Friday is a shopping holiday that takes ...,2,0,"[0.016857707872986794, -0.01078620832413435, -..."
3,"NOVEMBER 27, 2023",TIME,"November 27, 2023, is the date when Cyber Mond...",3,0,"[0.009768207557499409, -0.01335572637617588, -..."
4,AMAZON,ORGANIZATION,Amazon is a multinational technology company k...,4,1,"[0.029136504977941513, -0.007207231130450964, ..."
...,...,...,...,...,...,...
39353,ISRAEL-HAMAS WAR,EVENT,"A conflict involving Israel and Hamas, which h...",39353,4476,"[0.04042772203683853, -0.0035180281847715378, ..."
39354,EEA,LOCATION,The European Economic Area includes the Europe...,39354,4476,"[0.02691667526960373, -0.003410726087167859, -..."
39355,SWITZERLAND,LOCATION,A country in Europe that is part of the Europe...,39355,4476,"[0.015187944285571575, -0.011422696523368359, ..."
39356,DIGITAL SERVICES ACT,CONCEPT,The Digital Services Act is a regulation estab...,39356,4476,"[-0.010030446574091911, -0.01822492480278015, ..."


In [3]:
final_relationships=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_relationships_with_embeddings_by_chatgpt4o.csv",['relationship_embedding'])
final_relationships

,source_entity,target_entity,relationship_description,relationship_strength,source_entity_id,target_entity_id,corpus_id,relationship_embedding
0,AMAZON,CYBER MONDAY,Amazon is hosting numerous deals during Cyber ...,9,0,1,0,"[0.02443057671189308, -0.03799114748835564, -0..."
1,AMAZON,BLACK FRIDAY,Amazon began its Black Friday sale a week ahea...,8,0,2,0,"[0.054795797914266586, -0.031081540510058403, ..."
2,CYBER MONDAY,BLACK FRIDAY,"Cyber Monday follows Black Friday, with many d...",7,1,2,0,"[0.009650849737226963, -0.029358115047216415, ..."
3,"NOVEMBER 27, 2023",CYBER MONDAY,"November 27, 2023, is the date that marks the ...",10,3,1,0,"[0.007443748414516449, -0.01130700297653675, -..."
4,AMAZON,ECHO SHOW,"Amazon produces the Echo Show, integrating it ...",8,4,5,1,"[0.045543570071458817, -0.009272989816963673, ..."
...,...,...,...,...,...,...,...,...
29203,META,ISRAEL-HAMAS WAR,Meta's approach to content regulation is being...,6,39351,39353,4476,"[0.04234350472688675, -0.02529730089008808, -0..."
29204,META,DIGITAL SERVICES ACT,Meta is designated as a gatekeeper under the D...,8,39351,39356,4476,"[0.034957364201545715, -0.028356697410345078, ..."
29205,META,DIGITAL MARKETS ACT,Meta is also impacted by the Digital Markets A...,8,39351,39357,4476,"[0.010328758507966995, -0.016240786761045456, ..."
29206,EEA,META,"Meta serves users across the EEA, impacting it...",7,39354,39351,4476,"[0.034811727702617645, -0.027029171586036682, ..."


# Leiden/GNN聚类评估

In [4]:
from src.gnn_clustering.data_loader import load_custom_data
from src.gnn_clustering.utils import get_device
from tests.utils import test_model_performance
import numpy as np

# 设置设备
device = get_device()

# # 填充空的embedding
# expected_size = 1024
# final_entities['entity_embedding'] = final_entities['entity_embedding'].apply(
#     lambda x: np.zeros(expected_size) if x is None or len(x) == 0 else x
# )
# final_relationships['relationship_embedding'] = final_relationships['relationship_embedding'].apply(
#     lambda x: np.zeros(expected_size) if x is None or len(x) == 0 else x
# )
# # 去除异常边
# entity_ids = final_entities['id'].unique()
# final_relationships = final_relationships[
#     final_relationships['source_id'].isin(entity_ids) & final_relationships['target_id'].isin(entity_ids)
# ]


# 数据加载
data = load_custom_data(final_entities, final_relationships)
data = data.to(device)

# 执行测试
# report_df = test_model_performance(data=data, device=device, num_tests=3)

In [8]:
import pandas as pd

# 设置最大显示的列数为 None，表示不限制
pd.set_option('display.max_columns', None)
report_df=pd.read_csv("export/multihop_rag_report.csv")
report_df

,id,nodes,edges,initial_modularity,initial_clusters,initial_modularity_embedding,initial_clusters_embedding,avg_modularity_after_training,avg_clusters_after_training,avg_ami,improvement_leiden (%),improvement_embedding_leiden (%),head_1_modularity,head_1_num_clusters,head_2_modularity,head_2_num_clusters,head_3_modularity,head_3_num_clusters,head_1_ami_with_head_2,head_1_ami_with_head_3,head_2_ami_with_head_3
0,1,23570,52072,0.879755,7336,0.921488,243,0.936635,282.000000,0.067246,106.465507,101.643817,0.899519,272,0.951673,273,0.958713,301,0.055935,0.064235,0.081568
1,2,23570,52072,0.878740,7317,0.900986,237,0.940408,276.333333,0.073374,107.017731,104.375408,0.930744,292,0.947606,267,0.942874,270,0.076190,0.071583,0.072349
2,3,23570,52072,0.879584,7330,0.914862,253,0.948053,343.666667,0.085420,107.784256,103.627903,0.925607,310,0.969013,467,0.949538,254,0.089869,0.069830,0.096560
